<a href="https://colab.research.google.com/github/Shubham2376G/trainingIBM/blob/main/practice_finetuning_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer,TrainingArguments

In [68]:
!pip install datasets
from datasets import load_dataset, load_metric

In [69]:
pip install transformers[torch]

In [70]:
pip install accelerate -U

In [71]:
training_args=TrainingArguments(
    "mytraining",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64
)

In [72]:
# https://www.kaggle.com/crowdflower/twitter-airline-sentiment
!wget -nc https://lazyprogrammer.me/course_files/AirlineTweets.csv

File ‘AirlineTweets.csv’ already there; not retrieving.



In [73]:
import pandas as pd
import numpy as np

In [74]:
df= pd.read_csv('AirlineTweets.csv')

In [75]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [76]:
df=df[["airline_sentiment","text"]]

In [77]:
df.columns=(["label","sentence"])

In [78]:
df.head()

,label,sentence
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [103]:
df['label']=df['label'].map(target_map)

In [104]:
df.head()

,label,sentence
0,0,@VirginAmerica What @dhepburn said.
1,2,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,1,@VirginAmerica it's really aggressive to blast...
4,1,@VirginAmerica and it's a really big bad thing...


In [105]:
df.to_csv("data.csv",index=False)

In [106]:
raw_dataset=load_dataset("csv",
                         data_files="data.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [107]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence'],
        num_rows: 14640
    })
})

In [82]:
raw_dataset["train"][1:4]["sentence"]

["@VirginAmerica plus you've added commercials to the experience... tacky.",
 "@VirginAmerica I didn't today... Must mean I need to take another trip!",
 '@VirginAmerica it\'s really aggressive to blast obnoxious "entertainment" in your guests\' faces &amp; they have little recourse']

In [108]:
split=raw_dataset["train"].train_test_split(test_size=0.3,seed=5)
split

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence'],
        num_rows: 10248
    })
    test: Dataset({
        features: ['label', 'sentence'],
        num_rows: 4392
    })
})

In [84]:
checkpoint="distilbert-base-cased"

In [85]:
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [109]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'])

In [110]:
tokenized_datasets = split.map(tokenize_fn, batched=True)

Map:   0%|          | 0/10248 [00:00<?, ? examples/s]

Map:   0%|          | 0/4392 [00:00<?, ? examples/s]

In [88]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'sentence', 'input_ids', 'attention_mask'],
        num_rows: 10248
    })
    test: Dataset({
        features: ['label', 'sentence', 'input_ids', 'attention_mask'],
        num_rows: 4392
    })
})

In [89]:
model=AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [90]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

In [91]:
def compute_metric(logits_and_labels):
  logits, labels=logits_and_labels
  predictions=np.argmax(logits, axis=-1)
  acc=np.mean(predictions==labels)
  f1=f1_score(labels,predictions,average="macro")
  return({"acc":acc,"f1":f1})

In [92]:
from transformers import AutoConfig

In [93]:
config=AutoConfig.from_pretrained(checkpoint)

In [94]:
config

DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.40.2",
  "vocab_size": 28996
}

In [95]:
config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

In [96]:
target_map = {'positive': 2, 'negative': 1, 'neutral': 0}

In [97]:
config.id2label = {v:k for k, v in target_map.items()}
config.label2id = target_map

In [98]:
config.id2label

{2: 'positive', 1: 'negative', 0: 'neutral'}

In [99]:
config

DistilBertConfig {
  "_name_or_path": "distilbert-base-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "neutral",
    "1": "negative",
    "2": "positive"
  },
  "initializer_range": 0.02,
  "label2id": {
    "negative": 1,
    "neutral": 0,
    "positive": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.40.2",
  "vocab_size": 28996
}

In [111]:
trainer=Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

In [112]:
trainer.train()

Epoch,Training Loss,Validation Loss,Acc,F1
1,0.540300,0.464913,0.821038,0.746841
2,0.343400,0.469834,0.833333,0.777772


TrainOutput(global_step=1282, training_loss=0.4043756446302774, metrics={'train_runtime': 133.3266, 'train_samples_per_second': 153.728, 'train_steps_per_second': 9.615, 'total_flos': 259098433522848.0, 'train_loss': 0.4043756446302774, 'epoch': 2.0})

In [113]:
from transformers import pipeline

In [114]:
cls=pipeline("text-classification",model="mytraining/checkpoint-641")

In [121]:
cls(split["test"]["sentence"][5])

[{'label': 'LABEL_2', 'score': 0.5385791063308716}]

In [122]:
split["test"][5]

{'label': 2,
 'sentence': "@JetBlue it's okay...I'm actually a tea kinda guy anyway. Thanks!"}

In [1]:
from transformers import AutoTokenizer

In [4]:
tokenizer=AutoTokenizer.from_pretrained("Bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [8]:
tokenizer("hi , how are you","hey bro")

{'input_ids': [101, 20844, 117, 1293, 1132, 1128, 102, 23998, 9304, 1186, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}